> Igor Sorochan DSU-31

# Loss function and optimization problem implementation without sklearn

In [87]:
from sklearn import datasets
import numpy as np
import pandas as pd

import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, accuracy_score

np.random.seed(66)

### Loading data

In [88]:
iris = datasets.load_iris()
iris.feature_names, iris.target_names

(['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'],
 array(['setosa', 'versicolor', 'virginica'], dtype='<U10'))

In [89]:
print(iris.target_names)
# ~(iris.target_names == 'setosa')

['setosa' 'versicolor' 'virginica']


In [90]:
# masking as True 'versicolor' and 'virginica'
mask = iris.target > 0

In [91]:
X= iris.data[mask]
# reshape to assign dimensions
y= iris.target[mask].reshape(-1,1)
X.shape, y.shape

((100, 4), (100, 1))

### Visualizing input data

In [92]:
fig = px.scatter_matrix(X, color = y.ravel(), height= 900)
# fig.update_traces(diagonal_visible=False)
fig.show()

/Users/velo1/opt/anaconda3/lib/python3.9/site-packages/plotly/express/_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



### The least squares method in a matrix form

Given: a linear regression model as  **matrix X [n, p]** - observations,   **column vector [n, 1]** y as dependent variable.
*  n observations 
*  p independent variables. 

The least squares method seeks to find the p x 1 column vector β that minimizes the sum of the squared residuals e, which is given by:

e = y - Xβ

The least squares estimate of β is the value that minimizes the sum of squared residuals e' * e, which is equivalent to minimizing the Euclidean norm of e. We can express this in matrix form as:
$$ \beta = (X^TX)^{-1}X^TY $$
where:
* $\beta$ is the least squares estimate of the parameters.
* $X^T$ is the transpose of the X matrix.  
* $(X^T X)^{-1}$ is the inverse of the product of $X^T$ and $X$.  
* $y$ is the column vector of actual values of the dependent variable.

|$$ \beta = (X^TX)^{-1}X^TY $$|
|:---|

Let's write it in code:

In [93]:
class Lin_regr():
    def __init__(self):
        self.coeffs= None
        print('Linear regression model is initialized')

    def fit(self, X, y):
        # number of observations
        self.rows = X.shape[0]
        
        XX = X

        # add eye column to the right, to simplify matrix operations with initial bias(intercept or B0)
        XX = np.hstack((X, np.ones((self.rows, 1))))

        # calculate coeffs as defined earlier
        self.coeffs = np.linalg.inv(XX.T @ XX) @ XX.T @ y
        print('Linear regression model is fitted')
        return 
        
    def predict(self, X):

        XX = np.hstack((X, np.ones((self.rows, 1))))

        y_pred = XX @ self.coeffs

        return y_pred
    
    def coefficients(self):
        print('Linear regression coefficients:')
        return self.coeffs

In [94]:
lr = Lin_regr() # initialize a LR model
lr.fit(X, y) # fitting the model

Linear regression model is initialized
Linear regression model is fitted


In [95]:
# ravel arrays to confirm DataFrame input format, bias at the right as we add eye column in LR fit()
pd.DataFrame({'Coef':lr.coefficients().ravel()}, index= iris.feature_names + ['bias'] ) # print out LR coefficients

Linear regression coefficients:


,Coef
sepal length (cm),-0.196060
sepal width (cm),-0.307550
petal length (cm),0.384264
petal width (cm),0.682845
bias,0.581361


Defining a metric. Calculating the proportion ot correct answers among all outputs.

In [96]:
# ravel arrays to confirm DataFrame input format
df = pd.DataFrame({'Predicted':np.round(lr.predict(X).ravel()), 'GT': y.ravel()})
df[df.Predicted != df.GT]
df["Result"] = df.apply(lambda x: 'TP' if x.Predicted == x.GT else 'FP', axis= 1 )
TP,FP = df.Result.value_counts()

$Accuracy = \frac{TP}{TP + FP}$

In [97]:
print(f'Model accuracy :{TP / (TP + FP)}')

Model accuracy :0.97


### Test our metric with sklearn accuracy_score

In [98]:
accuracy_score(df.Predicted, y) == TP / (TP + FP)

True

In [99]:
mean_squared_error(df.Predicted, y)

0.03

Visualizing some feature relations.

In [100]:
fig =px.scatter(x=X[:,2], y=X[:,3], color= df.Predicted + df.GT*2, title= 'Visualizing the regression predictions')
fig.update_layout(yaxis_title="petal width (cm)",
                xaxis_title = 'petal length (cm)')
fig.show()

Two dots of different colors are the regression errors:  
* Blues - proper predictions of class1 
* Yellows - proper predictions of class2
* `Purple` - error prediction of class1
* `Orange` - error prediction of class2

### Gradient descent optimization method

Inverse matrix $(X^TX)^{-1}$ used in calculating column vector $\beta$ doesn't always exist.  
Moreover, as the number of observations and features in our model increases,  
calculating the inverse matrix may becomes **costly** in terms of the use of computing power resources.  
That is why we often need to rely on **empirical methods** to find the minimum of the loss function.  

One of very popular method is the **Stochastic Gradient Descent**.  
 It is a popular optimization algorithm used in machine learning to minimize the loss function of a model.  
 Instead of calculating the gradient of the entire dataset,  
 SGD updates the model parameters based on the gradient of a **random subset of the dataset**,  
 which is referred to as a **"mini-batch"**. 
 This makes the algorithm more efficient, especially when working with large datasets.  
 
 The **"stochastic"** part of the name comes from the fact that the **random selection** of the mini-batch  
 introduces some randomness into the optimization process,  
 which can help the algorithm escape local minimum and find a better overall solution.

### Gradient descent optimizations

In [101]:
def linear_expression(x):
    return 4 * x + 7

objects_num = 1000
X = np.linspace(-5, 5, objects_num)
y = linear_expression(X) + np.random.randn(objects_num) * 3
X = X[:, np.newaxis]
y = y[:, np.newaxis]

In [102]:
px.scatter(x=X.ravel(),y=y.ravel())

The folowing code implements different optimization methods

In [158]:
class GradientOptimiser(Lin_regr):
    def __init__(self, **kwargs):
        '''
        GradientOptimiser inherits some parents methods and attributes (predict and coeffs)
        '''
        super().__init__(**kwargs) 
        self.coeffs = None
        
        print('GradientOptimiser is initialized')

    def fit(self, X, y, lr=0.01, max_iter=100, method = 'gradient', nbatches = 1, gamma= 0.9):
        '''
        The folowing code implements different optimization methods:
        - gradient descent
        - SGD with mimi - batches
        - RMSProp
        '''
        self.rows, n_features = X.shape
        self.nbatches = nbatches
        self.method = method
        self.gamma = gamma
        self.mselosses = []     # reset previously calculated losses
        self.gt = 0

        if self.coeffs is None:
            # increase number of features to handle the logit b0 (bias)
            self.coeffs = np.zeros(n_features + 1)
            # self.coeffs = np.random.normal(size=n_features + 1)

        # add a column of ones
        XX = np.hstack((X, np.ones((self.rows, 1)))) 
        

        # THE DESCENT
        for i in range(max_iter):
            W= self.coeffs
            y_pred = self.predict(X) # 

            self.mselosses.append(mean_squared_error(y_pred, y))   # fix mse loss

            if  self.method == 'nag':  # Nesterov accelerated gradient   
                self.coeffs = W*gamma - (1-gamma) * lr * self.gradient(XX, y, y_pred)  
                # self.coeffs = gamma * W  - (1-gamma)*lr * self.gradient(XX, y, y_pred)  
            elif self.method == 'nesterov':  # Nesterov momentum
               
                                                                    # XX - gamma * w - the core of RMSProp
                                                                    # as we calculate gradient on shifted XX 
                self.coeffs = gamma * W - (1-gamma)*lr * self.gradient(XX - gamma * W, y, y_pred) 

            elif self.method == 'adagrad':
                self.gt += (self.gradient(XX, y, y_pred))**2
                self.coeffs = gamma * W - (1- gamma)*lr * self.gradient(XX, y, y_pred) / np.sqrt(self.gt +1e-8)

            elif self.method == 'rmsprop':
                self.gt = self.gt * gamma +  (1 - gamma) * (self.gradient(XX , y, y_pred))**2
                self.coeffs =  gamma * W - (1 - gamma)*lr * self.gradient(XX, y, y_pred) / np.sqrt(self.gt + 1e-8)
                # self.coeffs = gamma * W - (1 - gamma)*lr * self.gradient(XX, y, y_pred) / np.sqrt(self.gt + 1e-8)                
            else:
                #  learning rate  * gradient
                self.coeffs -= lr * self.gradient(XX, y, y_pred)

        return 


    def gradient(self, XX, y, y_pred ):

        if self.method == 'gradient':  # gradient descent over full data set
            gradient =  2 * XX * (y_pred[:, np.newaxis] - y) 

        else:
            # generate batches by random indexes 
            np.random.seed(42)
            i = np.random.choice(np.arange(self.rows), size= int(self.rows / self.nbatches), replace= False)            
            gradient =  2 * XX[i] * (y_pred[i][:, np.newaxis] - y[i])

        gradient = gradient.mean(axis=0) # averaging
        return gradient

    def get_mseloss(self):
        return self.mselosses    

Define a plot function.

In [104]:
def plot_losses(losses, title):
    fig =px.line(losses, title= title, height= 700, labels= 'Gradient descent')
    fig.update_layout(yaxis_title="Loss function",
        xaxis_title = 'Number of iterations')
    fig.show()

### Gradient Descent over full dataset

In [139]:
optimiser = GradientOptimiser()

losses= {}
for lr in np.logspace(-1, -4, 5):
  optimiser.coeffs = None
  optimiser.fit(X, y, lr = lr , max_iter=300, method = 'gradient')  

  losses.setdefault (lr, optimiser.get_mseloss())

for k,v in losses.items():
    print (f'Learning rate: {k:f}, MSE:{round(v[-1],3)}')
plot_losses(losses, title= 'Gradient Descent over full dataset. Loss function')

Linear regression model is initialized
GradientOptimiser is initialized
Learning rate: 0.100000, MSE:8.736
Learning rate: 0.017783, MSE:8.736
Learning rate: 0.003162, MSE:9.966
Learning rate: 0.000562, MSE:31.531
Learning rate: 0.000100, MSE:102.835


### Gradient descent with random batches (SGD)

In [140]:
optimiser = GradientOptimiser()

losses= {}
for nbatches in [1, 10, 30, 50]:

  optimiser.coeffs = None
  optimiser.fit(X, y, lr = 1e-2 , max_iter=300, method = 'sgd', nbatches= nbatches)

  losses.setdefault (nbatches, optimiser.get_mseloss())

for k,v in losses.items():
    # print (f'Number of batches: {k}, MSE:{round(min(v),3)}')
    print (f'Number of batches: {k}, MSE:{round(v[-1],3)}')
plot_losses(losses, title= 'Stochastic Gradient Descent. Loss function')

Linear regression model is initialized
GradientOptimiser is initialized
Number of batches: 1, MSE:8.736
Number of batches: 10, MSE:8.947
Number of batches: 30, MSE:8.748
Number of batches: 50, MSE:8.967


With number of **batches = 1**, SGD is actually a **Gradient Descent** on full dataset.  
As the number of batches increased so the stochastic behavoir of SGD becomes more obvious.  
In extreme case (batches = observations) the convergency level is higher.  The loss function may have found a local minima.

Let's play with parameters. I set very low learning rate and significantly increase the number of iterations.

### Nesterov Accelerated Gradient


<p style="align: center;"><img align=center src="https://tex.s2cms.ru/svg/%0Av_t%20%3D%20%5Cgamma%20v_%7Bt-1%7D%20%2B%20%5Ceta%20%5Cnabla_%5Ctheta%20J(%20%5Ctheta)%0A" height=50/></p>

<p style="align: center;"><img align=center src="https://tex.s2cms.ru/svg/%0A%5Ctheta%20%3D%20%5Ctheta%20-%20v_t%0A" height=50/></p>


In [159]:
optimiser = GradientOptimiser()
losses= {}
for gamma in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7,0.8, .9, 1]:
  optimiser.nbatches = nbatches
  # reset already learned coefficients 
  optimiser.coeffs = None
  # empirically chosen lr to differentiate the behavoir of loss function
  optimiser.fit(X, y, lr = 1e-2, max_iter=300, method = 'nag', nbatches= 1, gamma = gamma)
  # losses.setdefault (nbatches, optimiser.get_mseloss())
  losses.setdefault (gamma, optimiser.get_mseloss())

for k,v in losses.items():
    print (f'Gamma: {k}, MSE:{round(v[-1],3)}')
plot_losses(losses, title= 'Nesterov Accelerated Gradient. Loss function')

Linear regression model is initialized
GradientOptimiser is initialized
Gamma: 0, MSE:155.085
Gamma: 0.1, MSE:155.085
Gamma: 0.2, MSE:155.085
Gamma: 0.3, MSE:155.085
Gamma: 0.4, MSE:155.085
Gamma: 0.5, MSE:155.085
Gamma: 0.7, MSE:155.085
Gamma: 0.8, MSE:155.085
Gamma: 0.9, MSE:155.085
Gamma: 1, MSE:192.306


### Nesterov momentum

<p style="align: center;"><img align=center src="https://tex.s2cms.ru/svg/%0Av_t%20%3D%20%5Cgamma%20v_%7Bt-1%7D%20%2B%20%5Ceta%20%5Cnabla_%5Ctheta%20J(%20%5Ctheta%20-%20%5Cgamma%20v_%7Bt-1%7D%20)%0A" height=50/></p>

<p style="align: center;"><img align=center src="https://tex.s2cms.ru/svg/%0A%5Ctheta%20%3D%20%5Ctheta%20-%20v_t%0A" height=50/></p>


In [160]:
optimiser = GradientOptimiser()
losses= {}
for gamma in [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9]:
  optimiser.nbatches = nbatches
  # reset already learned coefficients 
  optimiser.coeffs = None
  # empirically chosen lr to differentiate the behavoir of loss function
  optimiser.fit(X, y, lr = 1e-3, max_iter=100, method = 'nesterov', nbatches= 1, gamma = gamma)
  # losses.setdefault (nbatches, optimiser.get_mseloss())
  losses.setdefault (gamma, optimiser.get_mseloss())
for k,v in losses.items():
  print (f'Gamma: {k}, MSE:{round(v[-1],3)}')
plot_losses(losses, title= 'Nesterov momentum. Loss function')

Linear regression model is initialized
GradientOptimiser is initialized
Gamma: 0.1, MSE:187.788
Gamma: 0.2, MSE:187.794
Gamma: 0.3, MSE:187.8
Gamma: 0.4, MSE:187.806
Gamma: 0.5, MSE:187.813
Gamma: 0.7, MSE:187.825
Gamma: 0.8, MSE:187.831
Gamma: 0.9, MSE:187.837


### Adaptive gradient

In [161]:
optimiser = GradientOptimiser()
losses= {}
for gamma in [0.1, 0.2, 0.3, 0.4, 0.5, 0.7,0.8, .9]:
  optimiser.nbatches = nbatches
  # reset already learned coefficients 
  optimiser.coeffs = None
  # empirically chosen lr to differentiate the behavoir of loss function
  optimiser.fit(X, y, lr = 1e-2, max_iter=100, method = 'adagrad', nbatches= 1, gamma = gamma)
  # losses.setdefault (nbatches, optimiser.get_mseloss())
  losses.setdefault (gamma, optimiser.get_mseloss())
for k,v in losses.items():
    print (f'Gamma: {k}, MSE:{round(v[-1],3)}')

plot_losses(losses, title= 'Adaptive gradient. Loss function')

Linear regression model is initialized
GradientOptimiser is initialized
Gamma: 0.1, MSE:192.225
Gamma: 0.2, MSE:192.224
Gamma: 0.3, MSE:192.224
Gamma: 0.4, MSE:192.224
Gamma: 0.5, MSE:192.224
Gamma: 0.7, MSE:192.224
Gamma: 0.8, MSE:192.223
Gamma: 0.9, MSE:192.22


### RMSProp (Route Mean Square Propogation)

In [162]:
optimiser = GradientOptimiser()
losses= {}
for gamma in [0.1, 0.2, 0.3, 0.4, 0.5, 0.7,0.8, .9]:
  optimiser.nbatches = nbatches
  # reset already learned coefficients 
  optimiser.coeffs = None
  # empirically chosen lr to differentiate the behavoir of loss function
  optimiser.fit(X, y, lr = 1e-2, max_iter=100, method = 'rmsprop', nbatches= 1, gamma = gamma)
  # losses.setdefault (nbatches, optimiser.get_mseloss())
  losses.setdefault (gamma, optimiser.get_mseloss())
for k,v in losses.items():
    print (f'Gamma: {k}, MSE:{round(v[-1],3)}')

plot_losses(losses, title= 'Route Mean Square Propogation. Loss function')

Linear regression model is initialized
GradientOptimiser is initialized
Gamma: 0.1, MSE:191.498
Gamma: 0.2, MSE:191.498
Gamma: 0.3, MSE:191.498
Gamma: 0.4, MSE:191.498
Gamma: 0.5, MSE:191.498
Gamma: 0.7, MSE:191.498
Gamma: 0.8, MSE:191.498
Gamma: 0.9, MSE:191.498


$\gamma$ 

For small $\gamma$ and relatively large $\eta$ , the momentum is so large that the loss function is unlikely to converge